# 🤖 HEGEMON - Human-in-the-Loop Debate

**Phase 2.3/2.4**: Interactive debate with review generation and feedback collection

---

## 📋 Jak to działa?

1. **Debata startuje** - Katalizator generuje tezę
2. **Checkpoint pojawia się** - Widzisz podsumowanie i możesz zareagować
3. **Podajesz feedback** - Approve/Revise/Reject
4. **Debata kontynuuje** - Agent uwzględnia Twój feedback
5. **Powtarzaj** aż do syntezy

**Checkpointy:** POST_THESIS → POST_EVALUATION → PRE_SYNTHESIS

## 1️⃣ Setup (Run Once)

In [1]:
# Importy
import sys
from pathlib import Path

# # Dodaj project root do path (jeśli potrzebne)
# project_root = Path.cwd()
# if str(project_root) not in sys.path:
#     sys.path.insert(0, str(project_root))

# Główne importy HITL
from hegemon.graph_hitl_v3 import (
    create_hegemon_graph_hitl_v3,
    run_debate_hitl_v3,
)
from hegemon.schemas_hitl import DebateStateHITL
from hegemon.hitl import InterventionMode
from hegemon.config import get_settings

print("✅ Imports successful!")

ImportError: cannot import name 'secretmanager' from 'google.cloud' (unknown location)

In [ ]:
# Weryfikuj konfigurację
settings = get_settings()

print(f"Configuration:")
print(f"  Katalizator: {settings.katalizator.model}")
print(f"  Sceptyk: {settings.sceptyk.model}")
print(f"  Gubernator: {settings.gubernator.model}")
print(f"  Syntezator: {settings.syntezator.model}")
print(f"\nDebate settings:")
print(f"  Consensus threshold: {settings.debate.consensus_threshold}")
print(f"  Max cycles: {settings.debate.max_cycles}")

## 2️⃣ Define Mission

In [ ]:
# Zdefiniuj swoją misję
MISSION = """
Design a comprehensive AI strategy for a mid-size e-commerce company.

Context:
- Company: 150 employees, $50M annual revenue
- Current tech stack: Shopify + basic analytics
- Budget: $500K annually for AI initiatives
- Timeline: 12 months

Requirements:
1. Improve customer experience (personalization, recommendations)
2. Reduce operational costs (automation, optimization)
3. GDPR compliant
4. Limited ML expertise in-house

Deliverables needed:
- Technology stack recommendations
- Phased implementation roadmap (Q1-Q4)
- Team structure and hiring plan
- Risk analysis
- Success metrics (KPIs)
"""

print("📋 Mission defined:")
print(MISSION)

## 3️⃣ Select Intervention Mode

**Modes:**
- `observer` - Pasywny (tylko 1 checkpoint na końcu) ⏱️ ~5 min
- `reviewer` - Standard (3 checkpointy) ⏱️ ~15 min ⭐ **RECOMMENDED**
- `collaborator` - Szczegółowy (wszystkie checkpointy + guidance) ⏱️ ~30 min

In [ ]:
# Wybierz tryb
MODE = "reviewer"  # Zmień na "observer" lub "collaborator"

print(f"✅ Selected mode: {MODE}")
print(f"\nExpected checkpoints:")
if MODE == "observer":
    print("  - PRE_SYNTHESIS (1x przed syntezą)")
elif MODE == "reviewer":
    print("  - POST_THESIS (po każdym Katalizatorze)")
    print("  - POST_EVALUATION (po Gubernatorze)")
    print("  - PRE_SYNTHESIS (przed Syntetatorem)")
else:
    print("  - All checkpoints + detailed guidance")

## 4️⃣ Run HITL Debate (Interactive!)

⚠️ **UWAGA:** Ta komórka zatrzyma się na każdym checkpoincie i poczeka na Twój feedback!

**Co zobaczysz:**
1. **Executive Summary** - Krótkie podsumowanie co się wydarzyło
2. **Key Points** - 3-5 najważniejszych faktów
3. **Important Claims** - Claims z niską pewnością (highlighted)
4. **Suggested Actions** - AI sugeruje co zrobić
5. **Full Output** - Pełna odpowiedź agenta
6. **Your Feedback** - Widgety do podjęcia decyzji

**Dostępne akcje:**
- ✅ **Approve** - Kontynuuj (agent zrobił dobrze)
- ✏️ **Request Revision** - Agent ma poprawić (dodaj guidance)
- ❌ **Reject** - Zakończ debatę (coś poszło bardzo źle)

In [ ]:
# 🚀 START DEBATE!
print("="*80)
print("🤖 Starting HITL Debate...")
print("="*80)
print("\n⏳ Please wait for first checkpoint...\n")

# Run debate (INTERACTIVE - będzie czekać na Twój input!)
final_state = run_debate_hitl_v3(
    mission=MISSION,
    intervention_mode=MODE,
)

print("\n" + "="*80)
print("✅ Debate Complete!")
print("="*80)

## 5️⃣ Review Results

In [ ]:
# Summary statistics
print("📊 DEBATE SUMMARY")
print("="*80)
print(f"Total cycles: {final_state.cycle_count}")
print(f"Total contributions: {len(final_state.contributions)}")
print(f"Human feedbacks provided: {len(final_state.human_feedback)}")
print(f"Revisions requested: {final_state.revision_count}")
print(f"Final consensus score: {final_state.current_consensus_score:.2f}")
print()

In [ ]:
# Feedback breakdown
if final_state.human_feedback:
    print("📝 YOUR FEEDBACK HISTORY")
    print("="*80)
    
    for i, feedback in enumerate(final_state.human_feedback, 1):
        print(f"\n{i}. Checkpoint: {feedback.checkpoint.value}")
        print(f"   Decision: {feedback.decision.value}")
        print(f"   Timestamp: {feedback.timestamp.strftime('%H:%M:%S')}")
        
        if feedback.guidance:
            print(f"   Your guidance: {feedback.guidance[:100]}...")
        
        if feedback.priority_claims:
            print(f"   Priority claims: {len(feedback.priority_claims)}")
        
        if feedback.flagged_concerns:
            print(f"   Flagged concerns: {len(feedback.flagged_concerns)}")

In [ ]:
# Final Plan overview
if final_state.final_plan:
    print("\n📋 FINAL PLAN OVERVIEW")
    print("="*80)
    
    plan = final_state.final_plan
    
    print(f"\nMission: {plan.mission_overview[:200]}...")
    print(f"\nRequired Agents: {len(plan.required_agents)}")
    for agent in plan.required_agents[:5]:  # Show first 5
        print(f"  - {agent.role}: {', '.join(agent.skills[:3])}")
    
    print(f"\nWorkflow Steps: {len(plan.workflow)}")
    for step in plan.workflow[:5]:  # Show first 5
        print(f"  {step.step_number}. {step.description[:60]}...")
        print(f"     Agent: {step.assigned_agent} | Duration: {step.estimated_duration}")
    
    print(f"\nTotal workflow duration: {plan.total_estimated_duration}")
    print(f"Risk score: {plan.overall_risk_score:.2f}")
else:
    print("\n⚠️ No final plan generated (debate may have been rejected)")

## 6️⃣ Save Results

In [ ]:
# Save to JSON
import json
from datetime import datetime

output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"hegemon_hitl_{MODE}_{timestamp}.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(
        final_state.model_dump(mode='json'),
        f,
        indent=2,
        ensure_ascii=False,
    )

print(f"💾 Results saved to: {output_file}")
print(f"   File size: {output_file.stat().st_size / 1024:.1f} KB")

## 7️⃣ Analyze Contributions (Optional)

In [ ]:
# Show all agent contributions
print("🔍 FULL DEBATE HISTORY")
print("="*80)

for contrib in final_state.contributions:
    print(f"\n{'='*80}")
    print(f"Agent: {contrib.agent_id} | Type: {contrib.type} | Cycle: {contrib.cycle}")
    print(f"{'='*80}")
    print(f"\n{contrib.content[:500]}...")
    print(f"\nRationale: {contrib.rationale[:200]}...")

## 8️⃣ Metrics & Analytics

In [ ]:
# Calculate metrics
from collections import Counter

print("📈 PERFORMANCE METRICS")
print("="*80)

# Intervention metrics
intervention_rate = len(final_state.human_feedback) / max(final_state.cycle_count, 1)
print(f"\nIntervention rate: {intervention_rate:.2f} feedbacks/cycle")

# Decision breakdown
if final_state.human_feedback:
    decisions = Counter(f.decision.value for f in final_state.human_feedback)
    total = len(final_state.human_feedback)
    
    print(f"\nDecision breakdown:")
    for decision, count in decisions.items():
        percentage = (count / total) * 100
        print(f"  {decision}: {count} ({percentage:.1f}%)")
    
    approval_rate = decisions.get('approve', 0) / total
    print(f"\nApproval rate: {approval_rate:.1%}")

# Revision metrics
revision_rate = final_state.revision_count / max(final_state.cycle_count, 1)
print(f"\nRevision rate: {revision_rate:.2f} revisions/cycle")

# Quality indicator
quality_score = final_state.current_consensus_score * approval_rate
print(f"\nQuality score: {quality_score:.2f} (consensus × approval rate)")

---

## 🎉 Done!

**Next steps:**
1. Review the final plan in detail
2. Export to your preferred format (PDF, Word, etc.)
3. Share with stakeholders
4. Run another debate with different intervention mode to compare

**Tips:**
- Use `observer` mode for quick iterations
- Use `reviewer` mode for production (best balance)
- Use `collaborator` mode for critical decisions
- Provide specific guidance in revisions for best results